In [31]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import requests
from queue import Queue, Empty
from concurrent.futures import ThreadPoolExecutor

In [52]:
class Crawler:
    def __init__(self, root):
        print("HI")
        protocol, host = urlparse(root).scheme, urlparse(root).netloc
        self.root = "{}://{}".format(protocol, host)
        self.explored = set()
        self.q = Queue()
        self.q.put(root)
        self.pool = ThreadPoolExecutor(max_workers=20)
    
    def scrape_page(self, url):
        try:
            res = requests.get(url, timeout=(3, 30))
            return res
        except requests.RequestException:
            return
    
    def post_scrape_call_back(self, res):
        result = res.result()
        if result and result.status_code == 200:
            self.parse_links(result.text)
    
    def parse_links(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all("a", href=True)
        for link in links:
            url = link['href']
            if url.startswith('/') or url.startswith(self.root):
                url = urljoin(self.root, url)
                if url not in self.explored:
                    self.q.put(url)
                
    def run(self):
        print("S")
        while True:
            try:
                task_url = self.q.get(timeout=60)
                if task_url not in self.explored:
                    print("Scraping URL: {}".format(task_url))
                    self.explored.add(task_url)
                    job = self.pool.submit(self.scrape_page, task_url)
                    job.add_done_callback(self.post_scrape_call_back)
            except Empty:
                return
            except Exception as e:
                print(e)
                continue
                

In [53]:
Crawler("http://www.sina.com").run()

HI
S
Scraping URL: http://www.sina.com
